In [2]:
# Coneccion entre Google Colab y el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **<font color="#ff5653">Funciones</font>**

En este notebook se creara el archivo donde las funciones tomaran la información.

Archivos utilizados:
 - steam_games_ETL.csv
 - users_items_ETL.csv
 - users_reviews_ETL.csv

Funciones que tomaran la información:
- Users_Recommend
- Users_Not_Recommend
- Sentiment_analysis

Archivos creados:
- df_f34y5.csv
- df_f34y5.parquet

In [45]:
# Libreria
import pandas as pd
import numpy as np
import re                     # Expresiones regulares

### 1- Extraemos el año

Del df_user_reviews extraemos el año de la columna "posted" y lo colocamos en una nueva columna.

In [46]:
# Abrimos el archivo user_reviews_sentimen_analysis.csv
df_user_reviews = pd.read_csv("//content/drive/MyDrive/Datasets_Steam/user_reviews_ETL.csv")

In [47]:
# Columna "posted", vemos si hay valores nulos
df_user_reviews["posted"].isna().sum()

0

In [48]:
df_user_reviews.shape

(59333, 9)

In [49]:
# Utiliza una expresión regular para buscar años en formato "YYYY" en cada elemento de la columna "posted".

lista_posted = df_user_reviews["posted"]
lista_almacenaje = []
lista_años = r'\b\d{4}\b'       # Busca año en formato "YYYY"

for fechas in lista_posted:
    # Si el elemento es una cadena, buscar un año en formato "YYYY"
    if isinstance(fechas, str):
        yyyy = re.search(lista_años, fechas)
        if yyyy:
            lista_almacenaje.append(yyyy.group())
        else:
            lista_almacenaje.append('NaN')
    # Si el elemento no es una cadena, agregar 'NaN' a la lista de almacenaje
    else:
        lista_almacenaje.append('NaN')

In [50]:
# Vemos lo que extrajimos
lista_almacenaje[:10]

['2011', '2011', '2011', '2014', '2013', '2013', 'NaN', '2015', '2014', '2014']

Para poder cocatenar la lista "lista_almacenaje" al dataframe df_user_review, debo tener la misma cantidad de filas.

De no tener las misma cantidad de filas los valores faltantes se rellenan con valores nulos.

En la "lista_almacenaje" aparecen "NaN", esto se debe que en la lista no poseen el año de publicado (ver ejemplos).

Y como el ultimo año de publicado es 2015, suponemos que estos datos fueron extraidos en 2016 por lo que donde hay "NaN" en "lista_almacenaje" voy a colocar 2016.

De esta forma al cocatenar "lista_almacenaje" a df_user_review, tendran las mismas cantidad de filas y evitamos que se agregen valores nulos automaticamente al dataframe.

Ejemplos:

- Sin año:
- indice=6 Posted February 3.
- indice=56 Posted August 23.

- Con año:
- indice=7 Posted December 4, 2015.
- indice=8 Posted November 3, 2014.
- indice=12 Posted October 14, 2013.
- indice=13 Posted July 28, 2012.
- indice=1 Posted July 15, 2011.

In [51]:
# Reemplazamos "NaN" por "2016"
lista_almacenaje_x = ["2016" if x == "NaN" else x for x in lista_almacenaje]

In [52]:
np.shape(lista_almacenaje_x)

(59333,)

In [53]:
np.shape(df_user_reviews)

(59333, 9)

In [54]:
# Concatenamos la lista al dataframe
df_user_reviews_x = pd.concat([df_user_reviews, pd.DataFrame({"año_publicado": lista_almacenaje_x})], axis=1)

In [55]:
df_user_reviews_x.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,analisis_sentimiento,año_publicado
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,True,2,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200,No ratings yet,True,2,2011


### 2- Extraemos el año

Del df_steam_games extraemos el año de la columna "release_date" y lo colocamos en una nueva columna.

In [56]:
# Abrimos el archivo steam_games_limpio.csv
df_steam_games = pd.read_csv("/content/drive/MyDrive/Datasets_Steam/steam_games_ETL.csv")

In [57]:
# Suma de valores nulos de la columna "release_date"
df_steam_games['release_date'].isna().sum()

2067

In [58]:
# Funcion para extraer el año
def año_lanzamiento(fecha):
    if pd.notna(fecha):
        años = re.search(r'\b\d{4}\b', fecha)
        if años:
            return años.group()
    return np.NaN

In [59]:
# Llamamos a la funcion: solo para la columna "release_date"
año_lanzamiento = df_steam_games["release_date"].apply(año_lanzamiento)

In [60]:
año_lanzamiento.head(5)

0    2018
1    2018
2    2017
3    2017
4     NaN
Name: release_date, dtype: object

In [61]:
# Concatenamos la columna "año_lanzamiento" con el DataFrame df_steam_games
df_steam_games_año = pd.concat([df_steam_games, pd.DataFrame({"año_lanzamiento": año_lanzamiento})], axis=1)

In [62]:
df_steam_games_año.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,año_lanzamiento
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro,2018
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,2018


### 3- Horas de juego por genero

De df_users_items sumamos las horas de juego de "Playtime_forever" agrupados por los ID de la columna "item_id" tendremos las horas de juego para cada genero.

In [63]:
# Abrimos el archivo users_items
df_users_items = pd.read_csv("/content/drive/MyDrive/Datasets_Steam/users_items_ETL.csv")

In [64]:
# Agrupamos los ID similar de las columnas "item_id" y "user_id", para despues se suman las horas de la columna "playtime_forever" de cada columna agrupada
columnas_df_users_items = df_users_items.groupby(["item_id", "user_id"])["playtime_forever"].sum().reset_index()

In [65]:
columnas_df_users_items.head(2)

,item_id,user_id,playtime_forever
0,10.0,-GM-Dragon,14.0
1,10.0,-KillZone-,987.0


In [66]:
# Unimos los dataframe columnas_df_users_items con df_user_reviews_x
df_items_y_reviews = pd.merge(columnas_df_users_items, df_user_reviews_x, on=["item_id", "user_id"], how="inner")

In [67]:
# Tenemos en un dataframe "item_id", la suma de "playtime_forever", agrupados por "user_id", el año en que el usuario jugo "año_publicado" y que hizo la reseña "analisis_sentimiento"
df_items_y_reviews.head(2)

,item_id,user_id,playtime_forever,user_url,funny,posted,last_edited,helpful,recommend,analisis_sentimiento,año_publicado
0,10.0,71251241,566.0,http://steamcommunity.com/id/71251241,NaN,"Posted November 7, 2015.",NaN,No ratings yet,True,2,2015
1,10.0,76561198015886143,144786.0,http://steamcommunity.com/profiles/76561198015...,NaN,"Posted June 14, 2015.",NaN,1 of 1 people (100%) found this review helpful,True,1,2015


In [68]:
# Cambiamos en df_steam_games_año el nombre de la columna a "id" a "item_id" para poder unir a otro dataframe por "item_id"
df_steam_games_año.rename(columns={"id": "item_id"}, inplace=True)

In [69]:
# Unimos los dataframe df_items_y_reviews con df_steam_games_año
df_xxx = pd.merge(df_items_y_reviews, df_steam_games_año, on="item_id", how='inner')

In [70]:
df_xxx.head(2)

,item_id,user_id,playtime_forever,user_url,funny,posted,last_edited,helpful,recommend,analisis_sentimiento,...,title,url,release_date,tags,reviews_url,specs,price,early_access,developer,año_lanzamiento
0,10.0,71251241,566.0,http://steamcommunity.com/id/71251241,NaN,"Posted November 7, 2015.",NaN,No ratings yet,True,2,...,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,2000
1,10.0,76561198015886143,144786.0,http://steamcommunity.com/profiles/76561198015...,NaN,"Posted June 14, 2015.",NaN,1 of 1 people (100%) found this review helpful,True,1,...,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,2000


In [71]:
df_xxx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40748 entries, 0 to 40747
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               40748 non-null  float64
 1   user_id               40748 non-null  object 
 2   playtime_forever      40748 non-null  float64
 3   user_url              40748 non-null  object 
 4   funny                 5874 non-null   object 
 5   posted                40748 non-null  object 
 6   last_edited           4225 non-null   object 
 7   helpful               40748 non-null  object 
 8   recommend             40748 non-null  object 
 9   analisis_sentimiento  40748 non-null  int64  
 10  año_publicado         40748 non-null  object 
 11  publisher             38720 non-null  object 
 12  genres                39071 non-null  object 
 13  app_name              40748 non-null  object 
 14  title                 39205 non-null  object 
 15  url                

### 4- Columnas requeridas para las funciones:

- item_id
- title
- año_lanzamiento
- genres
- user_id
- playtime_forever
- recomend
- analisis_sentimiento
- año_publicado

In [72]:
# Creamos columnas a extraer
columnas_x9 = ["item_id","title","año_lanzamiento","genres","user_id","playtime_forever","recommend","analisis_sentimiento","año_publicado"]

In [73]:
# Creamos nuevo dataframe con las columnas seleccionadas
df_f34y5 = df_xxx[columnas_x9]

In [ ]:
df_f34y5.head(2)

,item_id,title,año_lanzamiento,genres,user_id,playtime_forever,recommend,analisis_sentimiento,año_publicado
0,10.0,Counter-Strike,2000,['Action'],71251241,566.0,True,2,2015
1,10.0,Counter-Strike,2000,['Action'],76561198015886143,144786.0,True,1,2015


In [74]:
df_f34y5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40748 entries, 0 to 40747
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               40748 non-null  float64
 1   title                 39205 non-null  object 
 2   año_lanzamiento       39149 non-null  object 
 3   genres                39071 non-null  object 
 4   user_id               40748 non-null  object 
 5   playtime_forever      40748 non-null  float64
 6   recommend             40748 non-null  object 
 7   analisis_sentimiento  40748 non-null  int64  
 8   año_publicado         40748 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.1+ MB


In [75]:
# Cambiamos el tipo de datos de la columna "title" de object a string
df_f34y5["title"] = df_f34y5["title"].astype("string")

<ipython-input-75-3cb7de624e16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f34y5["title"] = df_f34y5["title"].astype("string")


In [80]:
# Cambiamos el tipo de datos de la columna "año_publicado" de object a int64
df_f34y5["año_publicado"] = df_f34y5["año_publicado"].astype("int64")

<ipython-input-80-3849888a5064>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_f34y5["año_publicado"] = df_f34y5["año_publicado"].astype("int64")


In [ ]:
# Eliminamos valores nulos de "año_lanzamiento" para poder cambiar el tipo de dato.
df_f34y5.dropna(subset="año_lanzamiento",inplace=True)

In [ ]:
# Cambiamos el tipo de datos de la columna "año_lanzamiento" de object a int64
df_f34y5["año_lanzamiento"] = df_f34y5["año_lanzamiento"].astype("int64")

In [87]:
df_f34y5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39149 entries, 0 to 40747
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_id               39149 non-null  float64
 1   title                 39149 non-null  string 
 2   año_lanzamiento       39149 non-null  int64  
 3   genres                39015 non-null  object 
 4   user_id               39149 non-null  object 
 5   playtime_forever      39149 non-null  float64
 6   recommend             39149 non-null  object 
 7   analisis_sentimiento  39149 non-null  int64  
 8   año_publicado         39149 non-null  int64  
dtypes: float64(2), int64(3), object(3), string(1)
memory usage: 3.0+ MB


In [88]:
df_f34y5.shape

(39149, 9)

In [89]:
# Guardamos el dataframe obtenido en formato .csv y parquet
df_f34y5.to_csv("df_f3f4yf5.csv", index=False)

In [90]:
df_f34y5.to_parquet("df_f3f4yf5.parquet", index=False)